#Taking out links from the list of apartment links

In [12]:
import requests
from bs4 import BeautifulSoup as bs

content_list = []

links = []

with open("apartments_links.txt", "r") as file:
  content = file.readlines()

  for line in content:
    links.append(line.replace("\n",""))

len(links)

20001

#Fetch the data for each feature for the data set methods

In [13]:
import concurrent.futures
import requests
from bs4 import BeautifulSoup as bs
import concurrent.futures

apartment_info = {}

list_of_apartments = []

def info_item(data_name, content):

  info = None

  soup = bs(content, "html.parser")

  div_structure = soup.find('div',{'class': 'offer__info-item', 'data-name': data_name})

  if div_structure != None:

    if data_name == "map.complex":

      info = div_structure.find("a").text.strip()

    else:

      info = div_structure.find('div', class_='offer__advert-short-info').text.strip()

  else:

    info = None

  return info

# --------------------------------------

def offer_description(data_name, content):

  info = None

  soup = bs(content, "html.parser")

  try:

    div_description = soup.find('div', class_='offer__description')

    description_list = div_description.find_all('dl')


    for description in description_list:


      data_name_temp = description.find("dt")

      if data_name_temp["data-name"].strip() == data_name:

        info = description.find("dd").text.strip()

  except Exception:

    info = None

  return info

# ---------------------------------------


def get_info(link):

  r = requests.get(link)

  content = r.content

  soup = bs(content, "html.parser")

  # Adding values into number_of_rooms, area, floor

  h1_list = soup.find("h1").text.strip().split(",")

  # (1) Number of rooms in apartment from heading
  number_of_rooms = h1_list[0]

  # (2) Price of the apartment
  try:

    price = soup.find("p", {"class": "offer__price"}).text.strip()

  except Exception:


    price = soup.find("div", {"class": "offer__price"})

    if price == None:

      price = None

    else:

      price = price.text.strip()

  # (3) Taking name of the district
  try:

    div_district = soup.find('div', class_='offer__location offer__advert-short-info')

    span_element_list = div_district.find('span').text.strip().split(",")

    district = span_element_list[1]

  except Exception:

    district = None


  # (4, 5, 6, 7, 8) Strutcure type , RC, year, floor, area, quality

  structure_type = info_item("flat.building", content)

  residential_complex = info_item("map.complex", content)

  year_of_construction = info_item("house.year", content)

  floor = info_item("flat.floor", content)

  area = info_item("live.square", content)

  quality = info_item("flat.renovation", content)

  # (9, 10) Bathroom, internet

  bathroom = offer_description("flat.toilet", content)

  internet_type = offer_description("inet.type", content)

  # Creating apartment with description

  apartment_info = {

      "number_of_rooms": number_of_rooms,
      "district": district,
      "structure_type": structure_type,
      "residential_complex": residential_complex,
      "year_of_construction": year_of_construction,
      "floor": floor,
      "area": area,
      "quality": quality,
      "bathroom": bathroom,
      "internet_type": internet_type,
      "price": price
  }

  # Appending apartments info to list
  print(apartment_info)

  list_of_apartments.append(apartment_info)


#Using for loop to fetch the information

In [14]:
links_modified = []

for i in range(16781, 17454):
  links_modified.append(links[i])

for link in links_modified:
  get_info(link)

print("done")

{'number_of_rooms': '1-комнатная квартира', 'district': ' Медеуский р-н', 'structure_type': 'монолитный', 'residential_complex': None, 'year_of_construction': '2020', 'floor': '5 из 9', 'area': '36.6 м², Площадь кухни — 12 м²', 'quality': 'хорошее', 'bathroom': None, 'internet_type': 'ADSL', 'price': '21\xa0600\xa0000\xa0〒'}
{'number_of_rooms': '3-комнатная квартира', 'district': ' Алмалинский р-н', 'structure_type': 'панельный', 'residential_complex': None, 'year_of_construction': '1974', 'floor': '2 из 5', 'area': '60 м², Площадь кухни — 6 м²', 'quality': 'хорошее', 'bathroom': None, 'internet_type': 'ADSL', 'price': '50\xa0000\xa0000\xa0〒'}
{'number_of_rooms': '3-комнатная квартира', 'district': ' Ауэзовский р-н', 'structure_type': 'панельный', 'residential_complex': None, 'year_of_construction': '1985', 'floor': '5 из 5', 'area': '61.7 м²', 'quality': 'хорошее', 'bathroom': None, 'internet_type': 'оптика', 'price': '45\xa0500\xa0000\xa0〒'}
{'number_of_rooms': '2-комнатная квартира'

#Checking data that was scraped and turn it into csv file

In [15]:
import pandas as pd

df = pd.DataFrame(list_of_apartments)

df


,number_of_rooms,district,structure_type,residential_complex,year_of_construction,floor,area,quality,bathroom,internet_type,price
0,1-комнатная квартира,Медеуский р-н,монолитный,None,2020,5 из 9,"36.6 м², Площадь кухни — 12 м²",хорошее,None,ADSL,21 600 000 〒
1,3-комнатная квартира,Алмалинский р-н,панельный,None,1974,2 из 5,"60 м², Площадь кухни — 6 м²",хорошее,None,ADSL,50 000 000 〒
2,3-комнатная квартира,Ауэзовский р-н,панельный,None,1985,5 из 5,61.7 м²,хорошее,None,оптика,45 500 000 〒
3,2-комнатная квартира,Ауэзовский р-н,монолитный,None,2007,5 из 10,"60 м², Площадь кухни — 11 м²",требует ремонта,None,ADSL,33 500 000 〒
4,1-комнатная квартира,Бостандыкский р-н,панельный,None,1983,2 из 5,"34.6 м², Площадь кухни — 11 м²",требует ремонта,None,None,33 000 000 〒
...,...,...,...,...,...,...,...,...,...,...,...
668,3-комнатная квартира,Наурызбайский р-н,кирпичный,None,2023,5 из 6,"53 м², Площадь кухни — 21 м²",хорошее,None,оптика,24 500 000 〒
669,2-комнатная квартира,Бостандыкский р-н,панельный,None,1979,5 из 5,"45 м², Площадь кухни — 6 м²",требует ремонта,None,None,35 000 000 〒
670,3-комнатная квартира,Наурызбайский р-н,монолитный,Elaman,2022,9 из 12,73.2 м²,хорошее,совмещенный,None,46 500 000 〒
671,2-комнатная квартира,Бостандыкский р-н,монолитный,Jazz-квартал,2023,8 из 16,"51 м², Площадь кухни — 23 м²",хорошее,совмещенный,оптика,54 900 000 〒


In [16]:
df.to_csv('almaty_apartments_partial1.csv', index=False)

#Combine part of scraped data to main csv file

In [18]:
import pandas as pd

# Load the first CSV file into a DataFrame
df1 = pd.read_csv('almaty_apartments_most.csv')

# Load the second CSV file into another DataFrame
df2 = pd.read_csv('almaty_apartments_partial1.csv')

# Concatenate the two DataFrames vertically (stack them on top of each other)
combined_df = pd.concat([df1, df2], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('almaty_apartments.csv', index=False)

combined_df


,number_of_rooms,district,structure_type,residential_complex,year_of_construction,floor,area,quality,bathroom,internet_type,price
0,3-комнатная квартира,Ауэзовский р-н,кирпичный,NaN,1996.0,1 из 5,"72 м², Площадь кухни — 12 м²",хорошее,NaN,через TV кабель,55 000 000 〒
1,3-комнатная квартира,Жетысуский р-н,монолитный,Maxima City,2024.0,NaN,63.06 м²,NaN,NaN,NaN,от 28 377 000 〒
2,7-комнатная квартира,Бостандыкский р-н,монолитный,Remizovka Life,2024.0,NaN,186.3 м²,NaN,NaN,NaN,от 158 355 000 〒
3,1-комнатная квартира,NaN,монолитный,New Life,2024.0,10 из 13,46.18 м²,NaN,NaN,NaN,20 845 220 〒
4,2-комнатная квартира,Ауэзовский р-н,монолитный,Уш Сункар,2020.0,9 из 10,43.4 м²,хорошее,совмещенный,ADSL,38 500 000 〒
...,...,...,...,...,...,...,...,...,...,...,...
19996,3-комнатная квартира,Наурызбайский р-н,кирпичный,NaN,2023.0,5 из 6,"53 м², Площадь кухни — 21 м²",хорошее,NaN,оптика,24 500 000 〒
19997,2-комнатная квартира,Бостандыкский р-н,панельный,NaN,1979.0,5 из 5,"45 м², Площадь кухни — 6 м²",требует ремонта,NaN,NaN,35 000 000 〒
19998,3-комнатная квартира,Наурызбайский р-н,монолитный,Elaman,2022.0,9 из 12,73.2 м²,хорошее,совмещенный,NaN,46 500 000 〒
19999,2-комнатная квартира,Бостандыкский р-н,монолитный,Jazz-квартал,2023.0,8 из 16,"51 м², Площадь кухни — 23 м²",хорошее,совмещенный,оптика,54 900 000 〒
